#**Моделирование диффузии частиц**
Этот проект я делал для доклада на своем факультете. Код ниже модделирует диффузиую газов двух разнородных незаряженных частиц, находящихся в закрытом объеме.

In [1]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import scipy as sp
import time
import cmath

In [2]:
def plot_surf(a,d):
    X = 3
    Y = np.arange(0, 3, 0.25)
    X, Y = np.meshgrid(X, Y)
    Z = np.arange(0, 3, 0.25)
    return X,Y,Z

In [4]:
def nach_znach(n_mol, raz, v_mean):
    r_1 = np.random.rand(n_mol[0], 3)
    r_1[:, 0]= raz[3]*r_1[:, 0]
    r_1[:, 1] = raz[1] * r_1[:, 1]
    r_1[:, 2] = raz[2] * r_1[:, 2]
    r_2 = np.random.rand(n_mol[1], 3)
    r_2[:, 0] = (raz[0]-raz[3]) * r_2[:,0] + raz[3]
    r_2[:, 1] = raz[1] * r_2[:, 1]
    r_2[:, 2] = raz[2] * r_2[:, 2]
    v_1 = np.random.normal(v_mean[0], 2*v_mean[0],(n_mol[0],3))
    v_2 = np.random.normal(v_mean[1], 2*v_mean[1],(n_mol[1],3))
    r = np.concatenate((r_1,r_2), axis = 0)
    v = np.concatenate((v_1, v_2), axis = 0)
    return r,v

In [5]:
def solve_quad(a, b, c):
    if a != 0:
        x1 = (-b + cmath.sqrt(b**2-4*c))/(2*a)
        x2 = (-b - cmath.sqrt(b**2-4*c))/(2*a)
        if abs(x2)<=abs(x1):
            p = x1
            x1 = x2
            x2 = p
        if abs(x1) == 0:
            x1 = c/x2
        elif abs(x2) == 0:
            x2 = c/x1
    else:
        x1 = -np.divide(c,b)
        x2 = x1
    return abs(x1), abs(x2)

In [14]:
def diff_func(m, n_mol, raz,r,v,t_0,t,toch,eps,sig):
    plt.ion()
    fig, ax = plt.subplots(subplot_kw=dict(projection='3d'))
    X = raz[3]
    Y = np.arange(0, raz[1], raz[1]/10)
    Z = np.arange(0, raz[2], raz[2]/ 10)
    Y,Z = np.meshgrid(Y,Z)
    ax.plot_surface(X,Y,Z,color= 'grey')
    r_1 = r[:n_mol[0], :]
    r_2 = r[n_mol[0]:n_mol[0]+n_mol[1], :]
    x, y, z = zip(*r_1)
    ax.scatter(x, y, z,color='green', s=20)
    x, y, z = zip(*r_2)
    ax.scatter(x, y, z, color='blue',s = 20)
    ax.view_init(30, -90)
    mng = plt.get_current_fig_manager()
    mng.full_screen_toggle()
    plt.show()
    plt.pause(1)
    plt.close('all')
    print('закрыт')
    l =  r.shape[0]
    iters = int((t - t_0)*toch*np.sqrt(max(sum(v[:,k]**2 for k in range(3))))/min(raz))
    print(iters)
    f = np.zeros(r.shape)
    dt = np.divide(t-t_0, iters)
    a = np.zeros(r.shape)
    for i in range(iters):
        if np.sum(f) !=0:
            print(i)
        for s in range(n_mol[0]):
            a[s][:] = np.divide(f[s][:], m[0])
        for s in range(n_mol[0], n_mol[0]+n_mol[1]):
            a[s][:] = np.divide(f[s][:], m[0])
        f = np.zeros(r.shape)
        for j in range(l):
            for s in range(3):
                if r[j][s] + v[j][s] * dt + np.divide(a[j][s] * dt ** 2, 2) >= raz[s]:
                    t_1, t_non = solve_quad(np.divide(a[j][s], 2), v[j][s], r[j][s] - raz[s])
                    r[j][s] = r[j][s] + v[j][s] * t_1 - (v[j][s] + a[j][s] * t_1) * (dt - t_1) + np.divide(a[j][s] * t_1 ** 2, 2)
                    v[j][s] = -v[j][s] - a[j][s] * t_1
                elif r[j][s] + v[j][s] * dt + np.divide(a[j][s] * dt ** 2, 2) <= 0:
                    t_1, t_non = solve_quad(np.divide(a[j][s], 2), v[j][s], r[j][s])
                    r[j][s] = r[j][s] + v[j][s] * t_1 - (v[j][s] + a[j][s] * t_1) * (dt - t_1) + np.divide(a[j][s] * t_1 ** 2,2)
                    v[j][s] = -v[j][s] - a[j][s] * t_1
                else:
                    r[j][s] = r[j][s] + v[j][s] * dt + np.divide(a[j][s] * dt ** 2, 2)
                    v[j][s] = v[j][s] + a[j][s] * dt
        for j in range(l):
            if j < n_mol[0]:
                energy = eps[0]
                sigma = sig[0]
                r_c = 2.5*sigma
            else:
                energy = eps[1]
                sigma = sig[1]
                r_c = 2.5*sigma
            for k in range(l):
                if sum(abs(r[k][p]**2-r[j][p]**2) for p in range(3))<=r_c**2 and k!=j:
                    R = np.sqrt(sum(abs(r[k][p]**2-r[j][p]**2) for p in range(3)))
                    for s in range(3):
                        f[k][s] +=24*energy*(sigma**12/R**13-sigma**6/R**7)*(r[j][s]-r[k][s])/R
        if i % 1 ==0:
            plt.close('all')
            fig, ax = plt.subplots(subplot_kw=dict(projection='3d'))
            ax.view_init(30, -90)
            mng = plt.get_current_fig_manager()
            mng.full_screen_toggle()
            X = raz[3]
            Y = np.arange(0, raz[1], raz[1] / 10)
            Z = np.arange(0, raz[2], raz[2] / 10)
            Y, Z = np.meshgrid(Y, Z)
            ax.plot_surface(X, Y, Z, color='grey')
            r_1 = r[:n_mol[0], :]
            r_2 = r[n_mol[0]:n_mol[0] + n_mol[1], :]
            x, y, z = zip(*r_1)
            ax.scatter(x, y, z, color='green', s=20)
            x, y, z = zip(*r_2)
            ax.scatter(x, y, z, color='blue', s=20)
            plt.show()
            plt.pause(0.01)

In [ ]:
N = 2
n_mol= np.array([100, 100], dtype=int)
a = np.float_power(n_mol[0]/10**26,1/3)
raz = [2*a, a,a,a]
v_mean = [478/np.sqrt(3), 500/np.sqrt(3)]
m = [5.3*1e-23,4.57*1e-26]
r, v = nach_znach(n_mol, raz, v_mean)
eps = np.array([99.22*1.38*1e-23, 95.5*1.38*1e-23])
sig = [3.52*1e-10,3.69*1e-10]
diff_func(m,n_mol,raz,r,v,0,0.01,10**2,eps,sig)